In [3]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [6]:
system_message = """
你是在方圓細胞生醫工作的櫃檯客服專員，請協助回答客戶的問題。
如果你不知道問題的答案，請誠實說「我不知道」，不要試圖編造答案。
"""

In [7]:
find_people = {"aurin": "Tech problem",
               "grace": "Lab problem", 
               "sam": "suicide prevention",
               "john": "Billing problem",
               "rat": "Micromanaging and do nothing",
               "carol": "Being an idiot and do nothing"}
def person_to_task(person):
    print(f"Tool called for person {person}")
    task_todo = find_people.get(person.lower(), "No such person found")
    return f"{person} is assigned to: {task_todo}"

In [9]:
find_function = {
    "name": "person_to_task",
    "description": "Track people and their assigned tasks",
    "parameters": {
        "type": "object",
        "properties": {
            "person": {
                "type": "string",
                "description": "The name of the person to find the assigned task for"
            }
        },
        "required": ["person"],
        "additionalProperties": False
    }
}

In [10]:
tools = [{"type": "function", "function": find_function}]

In [ ]:
available_functions = {"person_to_task": person_to_task}

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions.get(function_name)
        
        if function_to_call:
            # Parse arguments from JSON
            arguments = json.loads(tool_call.function.arguments)
            
            try:
                result = function_to_call(**arguments)
                content = str(result) if result is not None else f"Function {function_name} executed successfully"
                    
            except Exception as e:
                content = f"Error executing {function_name}: {str(e)}"
            
            responses.append({
                "role": "tool",
                "content": content,
                "tool_call_id": tool_call.id
            })
        else:
            responses.append({
                "role": "tool",
                "content": f"Error: Function {function_name} not found",
                "tool_call_id": tool_call.id
            })
    
    return responses

In [12]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool called for person aurin
Tool called for person carol
Tool called for person grace
Tool called for person rat
